In [4]:
qsrs.<q> = QQ[[]]

import copy

# We imported eta fractions from Conway-Norton and added constant terms as necessary.
# We use frame shape notation, for example 2 has symbol 1^24/2^24 corresponding
# to eta(t)^24/eta(2t)^24
eta_fractions =  {
    2 : ([(1,24)],[(2,24)], -24),
    3 : ([(1,12)],[(3,12)], -12),
    4 : ([(1,8)],[(4,8)], -8),
    5 : ([(1,6)], [(5,6)], -6),
    6 : ([(2,3),(3,9)],[(1,3),(6,9)], 3),
    7 : ([(1,4)],[(7,4)], -4),
    8 : ([(1,4), (4,2)],[(2,2),(8,4)], -4),
    9 : ([(1,3)],[(9,3)], -3),
    10 : ([(2,1),(5,5)],[(1,1),(10,5)], 1),
    12 : ([(3,3),(4,1)],[(1,1),(12,3)], 1),
    13 : ([(1,2)],[(13,2)], -2),
    16 : ([(1,2),(8,1)],[(2,1),(16,2)], -2),
    18 : ([(2,2), (9,1)],[(1,1),(18,2)], 1),
    25 : ([(1,1)],[(25,1)], -1),
}

def q_frac_pow_corr(l,m):
    """
    Sage's eta-fraction function doesn't include q^(1/24) in front.
    This function corrects that by taking in numerator and denominator
    of an eta fraction.
    """
    num_power = sum([a*b for (a,b) in l])
    den_power = sum([a*b for (a,b) in m])
    return 1/24*(num_power-den_power)

def princ_mod(level, n_terms):
    """
    Returns Hauptmodul of given level to n_terms of precision.
    """
    if level == 1:
        return j_invariant_qexp(n_terms-1)-744
    else:
        info = eta_fractions[level]
        nums = info[0]
        dens = info[1]
        const = info[2]
        t = qexp_eta(qsrs,n_terms)
        top = prod([(t(q^a))^n for (a,n) in nums])
        bottom = prod([(t(q^a))^n for (a,n) in dens])
        return (top/bottom*q^q_frac_pow_corr(nums,dens)-const)

def coef_list(qseries):
    """
    Gives list of coefficients of qseries, up qseries.prec()
    """
    coeftemp = qseries.list()
    for i in range(qseries.prec() - len(coeftemp)+1):
        coeftemp.append(0)
    return coeftemp

def integerize_matrix(mat, rows, cols):
    """
    Takes matrix with entries in e.g. finite field,
    along with dimensions, and outputs the matrix with
    the same entries but cast as integers.
    """
    newmat = []
    for i in range(rows):
        newrow = []
        for j in range(cols):
            newrow.append(int(mat[i][j]))
        newmat.append(copy.deepcopy(newrow))
    return newmat

def trim_relations(relations, l):
    """
    Takes relations, a list of lists of integers
    each representing a relation among Hauptmoduln with
    all entries after the l^th redundant (they are left 
    over because Sage has certain dimension restrictions
    when finding kernels which require padding with zeros).
    trim_relations cuts off the entries after the l^th
    for each entry in relations, then removes duplicates
    after this trimming.
    """
    newrelations = []
    for rel in relations:
        newrelations.append(tuple(rel[0:l]))
    return list(set(newrelations))

# NOTE: We need trim_relations because the following function uses a built-in
# function to find the kernel of a module map, which only works when the dimension
# of the codomain is at least the dimension of the domain. Hence, we pad the
# codomain and the matrix A with zeros in order to find ker(A), then remove
# these using trim_relations.

def find_kernel(A, numrows, numcols, p, a):
    """
    Finds the left kernel of matrix A of dimensions
    (numrows, numcols) in Z/p^aZ. Sage finds this by
    representing A (acting by left-multiplication)
    as a module homomorphism from (Z/p^aZ)^numrows
    to (Z/p^aZ)^numcols and computing its kernel.
    We require numrows <= numcols.
    """
    n = p**a
    newmat = copy.deepcopy(A)
    for i in range(0, numcols - numrows):
        # pad A with rows of zeros on bottom (for phi.kernel to work)
        newmat.append(numcols*[0])
    Zn = ZZ^numcols
    M = Zn/(n*Zn)
    
    # phi represents A as a module homomorphism
    phi = M.hom([M(a) for a in newmat])
    
    # find kernel of phi
    nontrimmed_relations = [M(b) for b in phi.kernel().gens()]
    
    # trims kernel elements to be elements of (Z/p^aZ)^numrows
    trimmed_relations =  trim_relations(nontrimmed_relations, numrows)
    
    return trimmed_relations

def is_new(relation, p):
    """
    Checks whether a relation comes from a lower power.
    """
    for coef in relation:
        if coef % p != 0:
            return True
    return False

def find_congs_mod(forms, sturm, p, power):
    """
    The parameter forms is a list of q-series (all taken to the same precision,
    equal to the sturm bound) to find congruences among, and sturm is the Sturm
    bound. Returns an error if a q-series has precision differing from the
    Sturm bound. This finds all relations among the q-series modulo p^power.
    Returns only those congruences which do not arise as p^a*Sigma for some linear
    combination Sigma of q-series which is zero modulo p^(n-a), for n,a < power.
    """
    numforms = len(forms)
    form_coefs = [] # this will become our matrix of coefficients
    for form in forms:
        formlist = tuple((coef_list(form)))
        if not len(formlist) == sturm:
            return 'Error: input form length was not equal to input sturm bound'
        form_coefs.append(formlist)
    newker = []
    ker = find_kernel(form_coefs, numforms, sturm, p, power)
    return ker

def find_all_congs(forms, sturm, p, check_bound):
    """
    The parameter forms is a list of q-series (all taken to the same precision,
    equal to the sturm bound) to find congruences among, and sturm is the Sturm
    bound. Returns an error if a q-series has precision differing from the
    Sturm bound. This finds all relations among the q-series modulo each power of p
    from 1 to check_bound, which it stores as a dictionary indexed by the modulus
    p^n of the congruence. Furthermore, the congruences stored at a given power
    p^n of p will be those which do not arise as p^a*Sigma for some linear
    combination Sigma of q-series which is zero modulo p^(n-a)
    """
    rel_dict = {}
    for i in range(1, check_bound):
        print 'power of p: ' + str(i)
        ker = find_congs_mod(forms, sturm, p, i)
        newker = []
        for rel in ker: # take all relations which are not lifts from a lower power
            if is_new(rel, p):
                newker.append(rel)
        if newker == []: # if all relations are lifts from lower power, stop checking powers
            break
        rel_dict[p^i] = copy.deepcopy(newker) # stores relations mod p^i
    return rel_dict

In [5]:
# Here we make a GUI to play with q-expansions of Hauptmoduln

@interact
def pri_q_exp_p_mod(level = [1] + eta_fractions.keys(), precision = slider(range(1,20))):
    print princ_mod(level,precision)
    return

In [6]:
# Here we give an example of listing all the congruences with
# T1(z) = j(z) - 744 and T7(z) = (eta(z)/eta(7z))^4 + 4

N = 10 # sturm bound will actually be 7, requiring a precision of 9

find_all_congs([princ_mod(1,N), princ_mod(7,N)], N, 7, 20)